In [1]:
import loompy
import os
import pandas as pd
import numpy as np
import re
import gc

/home/owenwhitley/anaconda3/envs/scvelo_conda/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9002. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
top_dir = "/home/owenwhitley/projects/su2c_v2"
data_dir = os.path.join(top_dir, 'data/preprocessed/scRNA/velocyto_GBM_GSCs_split_by_sample_no_G800_L')
merged_file = os.path.join(data_dir, 'GBM_GSCs_combined.loom')

In [3]:
merged_con = loompy.connect(merged_file)
merged_con.shape

(60675, 79862)

In [4]:
# Load DM coords
plot_data = pd.read_csv(os.path.join(data_dir, 'scvelo_plot_data_GBM_GSCs.csv'))
plot_data

,Unnamed: 0,nGene,nUMI,orig.ident,percent.mito,S.Score,G2M.Score,Phase,CC.Difference,Zhong_NPCs_upreg_AUC,...,Neftel_G2.M_AUC,PatientID,SampleID,Dev_IR_Diff,Stem_Astro_Diff,Dev_IR_Diff_zscore,Stem_Astro_Diff_zscore,DM1,DM2,unique_id
0,BTSC_BT127_L_ACGGAGACATCAGTAC,1317,3341,BTSC,0.222688,-0.162261,-0.063437,G1,-0.098824,0.064070,...,0.053853,BT127,BT127_L,0.204668,-0.136141,0.519910,-0.603678,-0.005989,-0.000526,BT127_L_ACGGAGACATCAGTAC
1,BTSC_BT127_L_GTGCATAAGCTGCCCA,1474,3245,BTSC,0.005547,0.467091,0.026003,S,0.441088,0.204442,...,0.073275,BT127,BT127_L,0.050891,0.093767,0.081005,0.487821,0.002134,-0.002152,BT127_L_GTGCATAAGCTGCCCA
2,BTSC_BT127_L_TCAGCAACAGCGTCCA,617,1343,BTSC,0.279226,0.049870,-0.133356,S,0.183226,0.093761,...,0.051747,BT127,BT127_L,0.283659,0.260084,0.745364,1.277425,-0.007759,-0.000145,BT127_L_TCAGCAACAGCGTCCA
3,BTSC_BT127_L_CTTAACTAGTTGAGAT,1016,1769,BTSC,0.054268,-0.108303,-0.230912,G1,0.122609,0.032523,...,0.001792,BT127,BT127_L,0.492503,0.118991,1.341437,0.607574,-0.006640,-0.000442,BT127_L_CTTAACTAGTTGAGAT
4,BTSC_BT127_L_CTGTTTACATGCGCAC,3526,13414,BTSC,0.088862,-0.170508,-0.286616,G1,0.116108,0.071696,...,0.039987,BT127,BT127_L,0.017085,-0.197140,-0.015482,-0.893279,0.002596,-0.003493,BT127_L_CTGTTTACATGCGCAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20338,GBM_G983-C_T_CTACACCCAAGGCTCC,4268,15647,GBM,0.045440,-0.136771,0.097905,G2M,-0.234676,0.152001,...,0.113418,G983-C,G983-C_T,0.286455,-0.311446,0.753341,-1.435953,-0.004245,-0.002802,G983-C_T_CTACACCCAAGGCTCC
20339,GBM_G983-C_T_ACTATCTTCAATAAGG,3053,6228,GBM,0.007868,0.645748,-0.132337,S,0.778086,0.136420,...,0.051680,G983-C,G983-C_T,0.480425,0.191462,1.306962,0.951638,-0.006172,-0.001402,G983-C_T_ACTATCTTCAATAAGG
20340,GBM_G983-C_T_GAGGTGATCCACGCAG,5064,24848,GBM,0.092281,-0.170040,-0.288269,G1,0.118229,0.071500,...,0.046595,G983-C,G983-C_T,0.108496,-0.661737,0.245421,-3.098984,-0.008866,-0.001709,G983-C_T_GAGGTGATCCACGCAG
20341,GBM_G983-C_T_TCGAGGCGTATCACCA,3348,10709,GBM,0.023905,-0.162106,-0.262224,G1,0.100117,0.063935,...,0.065636,G983-C,G983-C_T,0.018639,-0.560731,-0.011045,-2.619450,-0.013975,0.001115,G983-C_T_TCGAGGCGTATCACCA


In [5]:
plot_data['unique_id']

0         BT127_L_ACGGAGACATCAGTAC
1         BT127_L_GTGCATAAGCTGCCCA
2         BT127_L_TCAGCAACAGCGTCCA
3         BT127_L_CTTAACTAGTTGAGAT
4         BT127_L_CTGTTTACATGCGCAC
                   ...            
20338    G983-C_T_CTACACCCAAGGCTCC
20339    G983-C_T_ACTATCTTCAATAAGG
20340    G983-C_T_GAGGTGATCCACGCAG
20341    G983-C_T_TCGAGGCGTATCACCA
20342    G983-C_T_GCGGGTTAGATCCCAT
Name: unique_id, Length: 20343, dtype: object

In [6]:
new_loom_ids = merged_con.ca['CellID']

In [7]:
# filter cells not in diffusion map
in_DM = np.isin(new_loom_ids, plot_data['unique_id'].to_numpy())

In [8]:
sum(in_DM)

20343

In [9]:
len(plot_data)

20343

In [10]:
plot_data = plot_data.rename(index = plot_data['unique_id'])

In [11]:
# create subsetted loom. note that subsetting through loompy API is way too slow.
# Just load whole matrix into memory and subset
gc.collect()
subs_spl = merged_con.layers['spliced'][:, :]
subs_spl = subs_spl[:, in_DM]
gc.collect()
subs_uns = merged_con.layers['unspliced'][:, :]
subs_uns = subs_uns[:, in_DM]
gc.collect()

0

In [12]:
subs_ca = {}
subs_ca['CellID'] = merged_con.ca['CellID'][np.where(in_DM)]
for k in plot_data.columns:
    subs_ca[k] = plot_data[k].loc[subs_ca['CellID']].to_numpy()
# subs_ca.keys()

In [13]:
subs_file = os.path.join(data_dir, 'GSCs_GBM_combined_subs.loom')
if os.path.exists(subs_file):
    os.remove(subs_file)
loompy.create(subs_file, {'' : subs_spl, 'spliced' : subs_spl, 'unspliced' : subs_uns}, merged_con.ra, subs_ca)